In [8]:
from os import environ

environ['APRON_LD_PATH'] = '/Users/mazzu/.opam/qput/share/apron/lib'

from dataclasses import dataclass
from apronpy.var import PyVar

from apronpy.mpz import PyMPZ # multi-precision integer numbers
from apronpy.mpq import PyMPQ # multi-precision rational numbers
from apronpy.mpfr import PyMPFR # multi-precision floating-point numbers
from apronpy.coeff import PyDoubleScalarCoeff, PyMPQScalarCoeff, PyMPFRScalarCoeff, \
    PyDoubleIntervalCoeff, PyMPQIntervalCoeff, PyMPFRIntervalCoeff, PyScalarCoeff
from apronpy.scalar import PyDoubleScalar, PyMPQScalar, PyMPFRScalar
from apronpy.environment import PyEnvironment
from apronpy.linexpr1 import PyLinexpr1
from apronpy.lincons1 import PyLincons1, PyLincons1Array
from apronpy.lincons0 import ConsTyp
from apronpy.scalar import PyScalar
from apronpy.polka import PyPolkaMPQstrictManager, PyPolka
from apronpy.texpr1 import PyTexpr1
from apronpy.tcons1 import PyTcons1Array, PyTcons1

from functools import partial

from collections import defaultdict
from enum import Enum
from itertools import combinations

from tqdm import tqdm

from functools import partial


class Infix(object):
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        return self.func(other)
    def __ror__(self, other):
        return Infix(partial(self.func, other))
    def __call__(self, v1, v2):
        return self.func(v1, v2)

In [22]:
from enum import Enum

MyScalarCoeff = PyMPQScalarCoeff
# MyScalarCoeff = PyMPFRScalarCoeff

RightHandSide = PyVar | int
class OP(Enum):
  EQ = 0
  GE = 1
  GT = 2
  LE = 3
  LT = 4

  @property
  def to_apron(self) -> ConsTyp:
    match self:
      case OP.EQ:
        return ConsTyp.AP_CONS_EQ
      case OP.GE | OP.LE:
        return ConsTyp.AP_CONS_SUPEQ
      case OP.GT | OP.LT:
        return ConsTyp.AP_CONS_SUP

  @property
  def to_left_coeff(self) -> int:
    match self:
      case OP.EQ | OP.GE | OP.GT:
        return 1
      case OP.LE | OP.LT:
        return -1

  @property
  def to_rigth_coeff(self) -> int:
    return -self.to_left_coeff


class MyVar(PyVar):
  ENVIROMENT = PyEnvironment([])
  def __init__(self, name: str, is_real: bool = False):
    super().__init__(name)
    if is_real:
      MyVar.ENVIROMENT.add(real_vars=[self])
    else:
      MyVar.ENVIROMENT.add(int_vars=[self])

  def _create_constraint(
      self,
      op: OP,
      right: RightHandSide) -> PyTcons1:
    lexpr1 = PyLinexpr1(MyVar.ENVIROMENT)
    lexpr1.set_coeff(self, MyScalarCoeff(op.to_left_coeff))
    if isinstance(right, int):
      lexpr1.set_cst(MyScalarCoeff(op.to_rigth_coeff * right))
    elif isinstance(right, PyVar):
      lexpr1.set_coeff(right, MyScalarCoeff(op.to_rigth_coeff))
    else:
      raise TypeError('right must be a PyVar, PyScalarCoeff, or an int literal')

    lcons1 = PyLincons1(op.to_apron, lexpr1)
    tcons1 = PyTcons1(lcons1)
    return tcons1

  def as_expr(self) -> PyTexpr1:
    lexpr = PyLinexpr1(MyVar.ENVIROMENT)
    lexpr.set_coeff(self, MyScalarCoeff(1))
    texpr = PyTexpr1(lexpr)
    return texpr

  def __eq__(self, other: RightHandSide) -> PyTcons1:
    return self._create_constraint(OP.EQ, other)

  def __ge__(self, other: RightHandSide) -> PyTcons1:
    return self._create_constraint(OP.GE, other)

  def __gt__(self, other: RightHandSide) -> PyTcons1:
    return self._create_constraint(OP.GT, other)

  def __le__(self, other: RightHandSide) -> PyTcons1:
    return self._create_constraint(OP.LE, other)

  def __lt__(self, other: RightHandSide) -> PyTcons1:
    return self._create_constraint(OP.LT, other)

  @staticmethod
  def env() -> PyEnvironment:
    return MyVar.ENVIROMENT

def MyPolka(array: list[PyTcons1]) -> PyPolka:
  man = PyPolkaMPQstrictManager()
  env = MyVar.env()
  cons_array = PyTcons1Array(array, env)
  return PyPolka(man, env, array=cons_array)

class MyImplication:
  def __init__(self, left: PyPolka, right: PyPolka):
    self.left = left
    self.right = right

  def __str__(self) -> str:
    return f'({self.left}) => ({self.right})'

  def __repr__(self) -> str:
    return str(self)

  def join(self, other: "MyImplication") -> "MyImplication":
    left = self.left.meet(other.left)
    right = self.right.join(other.right)
    return MyImplication(left ,right)

  def widening(self, other: "MyImplication") -> "MyImplication":
    left = self.left.meet(other.left)
    if left != self.left:
      left = PyPolka.bottom(PyPolkaMPQstrictManager(), MyVar.env())
    right = self.right.widening(other.right)
    return MyImplication(left ,right)

  def is_top(self) -> bool:
    return self.left.is_bottom() and self.right.is_top()

  def is_bottom(self) -> bool:
    return self.left.is_top() or self.right.is_bottom()

  def __le__(self, other: "MyImplication") -> bool:
    return self.left <= other.left and other.right <= self.right

@Infix
def implies(self: PyPolka, other: PyPolka) -> MyImplication:
  return MyImplication(self, other)

class MyPartitions:
  K = 3
  def __init__(self, array: list[MyImplication]):
    self.array = []
    self.array = array

  def __str__(self) -> str:
    return '\n'.join([str(imp) for imp in self.array])

  def join(self, other: "MyPartitions") -> "MyPartitions":
    return MyPartitions(self.array + other.array)

def pprint_eqs(xs: list) -> str:
  if len(xs) == 0:
    return 'true'
  if len(xs) == 1:
    return "{ " + str(xs[0])
  higher = xs[0]
  middle = xs[1:-1]
  lower = xs[-1]
  out = "⎧ " + str(higher) + "\n"
  for i, x in enumerate(middle):
    if i == len(middle) // 2:
      out += "⎨ " + str(x) + "\n"
    else:
      out += "⎪ " + str(x) + "\n"
  out += "⎩ " + str(lower)
  return out

def minusminus(x: MyVar) -> PyTexpr1:
  lexpr = PyLinexpr1(MyVar.env())
  lexpr.set_coeff(x, MyScalarCoeff(1))
  lexpr.set_cst(MyScalarCoeff(-1))
  texpr = PyTexpr1(lexpr)
  return texpr

def plusplus(x: MyVar) -> PyTexpr1:
  lexpr = PyLinexpr1(MyVar.env())
  lexpr.set_coeff(x, MyScalarCoeff(1))
  lexpr.set_cst(MyScalarCoeff(1))
  texpr = PyTexpr1(lexpr)
  return texpr

def just_one(x: MyVar) -> PyTexpr1:
  lexpr = PyLinexpr1(MyVar.env())
  lexpr.set_coeff(x, MyScalarCoeff(1))
  texpr = PyTexpr1(lexpr)
  return texpr

a, b, c, d, x, y, z, r, r1, r2, r3, r4, i = [MyVar(name, is_real=True) for name in
  ['a', 'b', 'c', 'd', 'x', 'y', 'z', 'r', 'r1', 'r2', 'r3', 'r4', 'i']]
bottom = PyPolka.bottom(PyPolkaMPQstrictManager(), MyVar.env())
top = PyPolka.top(PyPolkaMPQstrictManager(), MyVar.env())

p = MyPolka([a >= 0, c == d])
q = p.forget([a])
print(q)
k = q.meet(MyPolka([a < 0]))
print(k)

p = p.join(k)
p


-1·c + 1·d + 0 == 0
-1·c + 1·d + 0 == 0 ∧ -1·a + 0 > 0


-1·c + 1·d + 0 == 0

In [5]:
@dataclass(frozen=True)
class A:

  def __post_init__(self):
    print("A")

@dataclass(frozen=True)
class B(A):
  def __post_init__(self):
    super().__post_init__()
    print("B")

@dataclass(frozen=True)
class C(B):
  def __post_init__(self):
    super().__post_init__()
    print("C")

x = C()

A
B
C


In [2]:
import tabulate
import itertools

def to_str(x):
  return "".join(map(str, x))

stream = list(itertools.product(range(3), repeat=5))
data = []

for ys in stream:
  print()
  for x in range(3):
    i = 0
    for y in ys:
      if y > x:
        break
      i += 1
    label = str(i)
    if i == 5:
      label = "unfinished"

    if label != "unfinished":
      print(x, to_str(ys), label)



0 00001 4

0 00002 4
1 00002 4

0 00010 3

0 00011 3

0 00012 3
1 00012 4

0 00020 3
1 00020 3

0 00021 3
1 00021 3

0 00022 3
1 00022 3

0 00100 2

0 00101 2

0 00102 2
1 00102 4

0 00110 2

0 00111 2

0 00112 2
1 00112 4

0 00120 2
1 00120 3

0 00121 2
1 00121 3

0 00122 2
1 00122 3

0 00200 2
1 00200 2

0 00201 2
1 00201 2

0 00202 2
1 00202 2

0 00210 2
1 00210 2

0 00211 2
1 00211 2

0 00212 2
1 00212 2

0 00220 2
1 00220 2

0 00221 2
1 00221 2

0 00222 2
1 00222 2

0 01000 1

0 01001 1

0 01002 1
1 01002 4

0 01010 1

0 01011 1

0 01012 1
1 01012 4

0 01020 1
1 01020 3

0 01021 1
1 01021 3

0 01022 1
1 01022 3

0 01100 1

0 01101 1

0 01102 1
1 01102 4

0 01110 1

0 01111 1

0 01112 1
1 01112 4

0 01120 1
1 01120 3

0 01121 1
1 01121 3

0 01122 1
1 01122 3

0 01200 1
1 01200 2

0 01201 1
1 01201 2

0 01202 1
1 01202 2

0 01210 1
1 01210 2

0 01211 1
1 01211 2

0 01212 1
1 01212 2

0 01220 1
1 01220 2

0 01221 1
1 01221 2

0 01222 1
1 01222 2

0 02000 1
1 02000 1

0 02001 1
1 02